## Prerequisites

In [4]:
# %pip install tabulate

## Extract configs of all stages as list of Dataframes

In [ ]:
import pandas as pd
from functools import reduce
from ska_sdp_spectral_line_imaging import pipeline
from ska_sdp_piper.piper.configurations.nested_config import NestedConfigParam

# def process_config_param(prefix, config_param):
#     if config_param._type is NestedConfigParam:
#         output = []
#         for name, inner_config in config_param._config_params.items():
#             output.extend(process_config_param(f"{prefix}.{name}", inner_config))
#         return output

#     return [{"param": prefix, **config_param.__dict__}]

def process_config_param_reduced(prefix, config_param):
    if config_param._type is NestedConfigParam:
        return reduce(lambda acc, param : [*acc, *process_config_param(f"{prefix}.{param[0]}", param[1])], config_param._config_params.items(), [])

    return [{"param": prefix, **config_param.__dict__}]


dfs = {}

for x in pipeline.spectral_line_imaging_pipeline._stages:
    df = []
    for name, config_param in x._Stage__config._config_params.items():
        df.extend(process_config_param_reduced(name, config_param))
    df = pd.DataFrame(df).fillna("None")
    if df.empty:
        continue
    df = df.rename(columns={'_type': 'type'})
    df = df.rename(columns={'_ConfigParam__value': 'default'})
    df = df.rename(columns={'allowed_values': 'allowed values'})
    df.columns = df.columns.str.capitalize()
    df['Type'] = df['Type'].apply(lambda x: x.__name__)
    df['Allowed values'] = df['Allowed values'].apply(lambda x: "" if x == "None" else x)
    dfs[x.name]=df

### For testing

In [6]:
def df_to_org_table(df):
    return df.to_markdown(index=False, tablefmt='grid', colalign=['left']*len(df.columns), maxcolwidths=[None, None, 40, 80])

In [ ]:
df = dfs["imaging"]
print(df_to_org_table(df))

## Generate the RST file

In [ ]:
header = """Stage Configs
=============

.. This file is generated using scripts/generate_config.py

.. This file is referenced by "imaging" stage docstring by a relative reference
.. to the generated html page.
"""

table_config = """
..  table::
    :width: 100%
    :widths: 15, 10, 10, 45, 10, 10
"""

indent = "    "

In [9]:
def generate_stage_config(dfs, stage_config_path):
    with open(stage_config_path, "w") as f:
        # Write the header first
        f.write(header)
        f.write("\n\n")

        for name, df in dfs.items():
            f.write(name)
            f.write("\n")
            f.write("*" * len(name))
            f.write("\n")
            f.write(table_config)
            f.write("\n")
            # Convert DataFrame to markdown string and write it to file
            markdown = df.to_markdown(index=False, tablefmt='grid', colalign=['left']*len(df.columns), maxcolwidths=[None, None, 40, 80])
            indented_markdown = "\n".join(indent + line for line in markdown.splitlines())

            f.write(indented_markdown)
            f.write("\n\n\n")  # Add some space between tables

In [10]:
generate_stage_config(dfs, "../docs/src/stage_config.rst")